In [ ]:
import wandb

In [ ]:
project_name = "isotropic_chi_v2"
run_name = "different-aardvark-14"

In [ ]:
# Initialize the API
api = wandb.Api()

# Retrieve the run (replace with your project name and run ID)
run = api.run("jack-smithy-university-of-vienna/isotropic_chi_v2/fjk37ycy")


In [ ]:
# Add the image or plot (e.g., a saved file)
run.log(
    {
        "histogram": wandb.Image(
            f"../results/{project_name}/{run_name}/hist_isotropic.png"
        ),
        "heatmap_amplitude": wandb.Image(
            f"../results/{project_name}/{run_name}/heatmap_amplitude_isotropic.png"
        ),
        "heatmap_angle": wandb.Image(
            f"../results/{project_name}/{run_name}/heatmap_angle_isotropic.png"
        ),

    }
)

# Save the updates
run.update()